# ee

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 20, "display.max_columns", 30)
import gc
import time
import os
import re
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
mini = 1

In [3]:
filenames = []
for dirname, _, filenamess in os.walk('home-credit-default-risk/'):
    for filenamee in filenamess:
        if filenamee != "HomeCredit_columns_description.csv":
            filename = os.path.join(dirname, filenamee)
            print(filename)
            filenames.append(filename)
            df = pd.read_csv(filename)
            display(df[-1:])
            if filenamee != "bureau_balance.csv":
                print(set(df.SK_ID_CURR.value_counts()))

home-credit-default-risk/application_test.csv


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
48743,456250,Cash loans,F,Y,N,0,135000.0,312768.0,24709.5,270000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,...,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0


{1}
home-credit-default-risk/application_train.csv


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,...,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
307510,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,675000.0,Unaccompanied,Commercial associate,Higher education,Married,...,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,2.0,0.0,1.0


{1}
home-credit-default-risk/bureau.csv


...

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 202, 203, 204, 205, 206, 207, 208, 211, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 225, 226, 227, 22

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,...,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
1670213,2418762,261212,Cash loans,16431.3,360000.0,360000.0,NaN,360000.0,SUNDAY,10,Y,1,NaN,NaN,NaN,...,XNA,Cash,x-sell,AP+ (Cash loan),-1,XNA,48.0,middle,Cash X-Sell: middle,365243.0,-1163.0,247.0,-443.0,-423.0,0.0


{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 72, 73, 77}
home-credit-default-risk/sample_submission.csv


,SK_ID_CURR,TARGET
48743,456250,0.5


{1}


In [4]:
%%script echo skipped
for filename in (filenames[:4] + filenames[5:]):
    df = pd.read_csv(filename)
    print(filename)
    display(df.head(2))
    #display(df.info())
    print("shape: ", df.shape)
    display(df.nunique())
    #display(df.isna().mean().sort_values())
    print("")

Couldn't find program: 'echo'


In [5]:
#pd.read_csv(filenames[4])

In [6]:
@contextmanager
def timer(title):
    t0 = time.perf_counter()
    yield
    print("{} - done in {:.0f}s".format(title, time.perf_counter() - t0))
    
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [7]:
df_ohe_test = pd.DataFrame({
    'C': ['R', 'G', 'B', 'B', 'R'],
    'L': ['A', 'B', 'C', 'D', 'A']
})

display(one_hot_encoder(df_ohe_test, False)[1])
display(one_hot_encoder(df_ohe_test, True)[0])

['C_B', 'C_G', 'C_R', 'L_A', 'L_B', 'L_C', 'L_D']

,C_B,C_G,C_R,C_nan,L_A,L_B,L_C,L_D,L_nan
0,0,0,1,0,1,0,0,0,0
1,0,1,0,0,0,1,0,0,0
2,1,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0,1,0
4,0,0,1,0,1,0,0,0,0


In [8]:
1234567890123456789012345678901234567890123456789012345678901234567890123456789

1234567890123456789012345678901234567890123456789012345678901234567890123456789

In [9]:
%%script echo skipped
num_rows = None
nan_as_category = False
df = pd.read_csv('../input/home-credit-default-risk/application_train.csv', nrows=num_rows)
test_df = pd.read_csv('../input/home-credit-default-risk/application_test.csv', nrows=num_rows)
print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
df = df.append(test_df).reset_index()
display(df.head(2))

Couldn't find program: 'echo'


In [10]:
%%script echo skipped
pd.factorize(df['CODE_GENDER'])

Couldn't find program: 'echo'


In [11]:
# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows=None, nan_as_category=False):
    # Read data and merge
    if mini:
        df = pd.read_csv(filenames[1], nrows=num_rows)[-10:]
        test_df = pd.read_csv(filenames[0], nrows=num_rows)[-2:]
    else:
        df = pd.read_csv(filenames[1], nrows=num_rows)
        test_df = pd.read_csv(filenames[0], nrows=num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    del test_df
    gc.collect()
    #print(df.dtypes.value_counts())
    return df

In [12]:
%%script echo skipped
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv', nrows=num_rows)
bb = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv', nrows=num_rows)
bb, bb_cat = one_hot_encoder(bb, nan_as_category)
bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
for col in bb_cat:
    bb_aggregations[col] = ['mean']
print(bb_aggregations, "\n", bb.columns, "\n", bureau_cat, "\n", bureau.columns)

Couldn't find program: 'echo'


In [13]:
%%script echo skipped
bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)
bureau.head(2)

Couldn't find program: 'echo'


In [14]:
%%script echo skipped
num_aggregations = {
    'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
    'DAYS_CREDIT_UPDATE': ['mean'],
    'CREDIT_DAY_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
    'AMT_ANNUITY': ['max', 'mean'],
    'CNT_CREDIT_PROLONG': ['sum'],
    'MONTHS_BALANCE_MIN': ['min'],
    'MONTHS_BALANCE_MAX': ['max'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum']
}
# Bureau and bureau_balance categorical features
cat_aggregations = {}
for cat in bureau_cat: cat_aggregations[cat] = ['mean']
for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']

bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
# Bureau: Active credits - using only numerical aggregations
active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
active.head(2)

Couldn't find program: 'echo'


In [15]:
# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows=None, nan_as_category=True):
    if mini:
        bureau = pd.read_csv(filenames[2], nrows=num_rows)[-5:-3]
        bb = pd.read_csv(filenames[3], nrows=num_rows)[-5:-3]
    else:
        bureau = pd.read_csv(filenames[2], nrows=num_rows)
        bb = pd.read_csv(filenames[3], nrows=num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    #print(bureau_agg.dtypes.value_counts())
    return bureau_agg

In [16]:
%%script echo skipped
df = pd.read_csv(filenames[2])[-5:]
df

Couldn't find program: 'echo'


In [17]:
%%script echo skipped
prev.columns

Couldn't find program: 'echo'


In [18]:
# Preprocess previous_applications.csv
def previous_applications(num_rows=None, nan_as_category=True):
    if mini:
        prev = pd.read_csv(filenames[7], nrows=num_rows)[-7:-5]
    else:
        prev = pd.read_csv(filenames[7], nrows=num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category=True)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    #print(prev_agg.dtypes.value_counts())
    return prev_agg

In [19]:
%%script echo skipped
df = pd.read_csv(filenames[7])[-7:]
df.NAME_CONTRACT_STATUS

Couldn't find program: 'echo'


In [20]:
%%script echo skipped
df.columns

Couldn't find program: 'echo'


In [21]:
# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows=None, nan_as_category=True):
    if mini:
        pos = pd.read_csv(filenames[6], nrows=num_rows)[-2:]
    else:
        pos = pd.read_csv(filenames[6], nrows=num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category=True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    #print(pos_agg.dtypes.value_counts())
    return pos_agg

In [22]:
# Preprocess installments_payments.csv
def installments_payments(num_rows=None, nan_as_category=True):
    if mini:
        ins = pd.read_csv(filenames[5], nrows=num_rows)[-2:]
    else:
        ins = pd.read_csv(filenames[5], nrows=num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category=True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    #print(ins_agg.dtypes.value_counts())
    return ins_agg

In [23]:
# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows=None, nan_as_category=True):
    if mini:
        cc = pd.read_csv(filenames[4], nrows=num_rows)[-2:]
    else:
        cc = pd.read_csv(filenames[4], nrows=num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category=True)
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis=1, inplace =True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    #print(cc_agg.dtypes.value_counts())
    return cc_agg

In [24]:
# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, num_folds, stratified=False, debug=False):
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits=num_folds, shuffle=True, random_state=1001)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1,
        )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric='auc', verbose=200, early_stopping_rounds=200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        test_df['TARGET'] = sub_preds
        test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index=False)
    display_importances(feature_importance_df)
    return feature_importance_df

In [25]:
# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature")\
           .mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')

In [26]:
def main(debug=False):
    num_rows = 1000 if debug else None
#    mini = 0 if debug else non sinon il est jamais full le df
#    if debug: mini = 0 En fait même celui-ci ne fontionne pas car lors de la
#    def des fcts mini valait 1 et lors le l'appel des fcts soit le "if mini"
#    ne sera pas lu car il a déjà été évalué lors du def ou alors le "if mini"
#    est lu mais ne regarde pas la valeur actuelle de mini seulement celle qui
#    était active lors du def. Je ne sais pas exactement mais le résultat est
#    que c'est trop tard ici pour dire "mini = 0".
    df = application_train_test(num_rows)
    print("Application train test df shape:", df.shape)
#    print(df.dtypes.value_counts())
    with timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(num_rows)
        print("Bureau df shape:", bureau.shape)
        df = df.join(bureau, how='left', on='SK_ID_CURR')
#        print(df.dtypes.value_counts())
        del bureau
        gc.collect()
    with timer("Process previous_applications"):
        prev = previous_applications(num_rows)
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how='left', on='SK_ID_CURR')
#        print(df.dtypes.value_counts())
        del prev
        gc.collect()
    with timer("Process POS-CASH balance"):
        pos = pos_cash(num_rows)
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how='left', on='SK_ID_CURR')
#        print(df.dtypes.value_counts())
        del pos
        gc.collect()
    with timer("Process installments payments"):
        ins = installments_payments(num_rows)
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how='left', on='SK_ID_CURR')
#        print(df.dtypes.value_counts())
        del ins
        gc.collect()
    with timer("Process credit card balance"):
        cc = credit_card_balance(num_rows)
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')
#        print(df.dtypes.value_counts())
        del cc
        gc.collect()
#    with timer("Run LightGBM with kfold"):
#        feat_importance = kfold_lightgbm(df, num_folds=10, stratified=False, debug=debug)
    return df

In [27]:
1234567890123456789012345678901234567890123456789012345678901234567890123456789

1234567890123456789012345678901234567890123456789012345678901234567890123456789

# ee

In [28]:
%%script echo skipped
# I ran this cell only once, just to get the exact values of zo and oz.
if __name__ == "__main__":
    submission_file_name = "submission_kernel02.csv"
    with timer("Full model run"):
#        main()
        df = main(debug=False)

zeros_full = df.TARGET.value_counts(
    sort=True,
    ascending=False,
    dropna=True,
)[0]
ones_full = df.TARGET.value_counts(
    sort=True,
    ascending=False,
    dropna=True,
)[1]
print(f"{zeros_full:.2f}", f"{ones_full:.2f}")

Couldn't find program: 'echo'


In [29]:
zeros_full, ones_full = 282682, 24825

In [30]:
zo = zeros_full/ones_full
oz = ones_full/zeros_full
print(f"{zo:.2f}", f"{oz:.2f}")

11.39 0.09


In [31]:
%%script echo
if __name__ == "__main__":
    submission_file_name = "submission_kernel02.csv"
    with timer("Full model run"):
#        main()
        df = main(debug=True)

zeros = df.TARGET.value_counts(
    sort=True,
    ascending=False,
    dropna=True,
)[0]
ones = df.TARGET.value_counts(
    sort=True,
    ascending=False,
    dropna=True,
)[1]
print(f"{zeros:.2f}", f"{ones:.2f}")

Couldn't find program: 'echo'


In [32]:
if __name__ == "__main__":
    submission_file_name = "submission_kernel02.csv"
    with timer("Full model run"):
        debug = True
        num_rows = 1000 if debug else None
#        mini = 0 if debug else non sinon il est jamais full le df
        if debug: mini = 0
        df = application_train_test(num_rows)
        print("Application train test df shape:", df.shape)
#        print(df.dtypes.value_counts())
        with timer("Process bureau and bureau_balance"):
            bureau = bureau_and_balance(num_rows)
            print("Bureau df shape:", bureau.shape)
            df = df.join(bureau, how='left', on='SK_ID_CURR')
#            print(df.dtypes.value_counts())
            del bureau
            gc.collect()
        with timer("Process previous_applications"):
            prev = previous_applications(num_rows)
            print("Previous applications df shape:", prev.shape)
            df = df.join(prev, how='left', on='SK_ID_CURR')
#            print(df.dtypes.value_counts())
            del prev
            gc.collect()
        with timer("Process POS-CASH balance"):
            pos = pos_cash(num_rows)
            print("Pos-cash balance df shape:", pos.shape)
            df = df.join(pos, how='left', on='SK_ID_CURR')
#            print(df.dtypes.value_counts())
            del pos
            gc.collect()
        with timer("Process installments payments"):
            ins = installments_payments(num_rows)
            print("Installments payments df shape:", ins.shape)
            df = df.join(ins, how='left', on='SK_ID_CURR')
#            print(df.dtypes.value_counts())
            del ins
            gc.collect()
        with timer("Process credit card balance"):
            cc = credit_card_balance(num_rows)
            print("Credit card balance df shape:", cc.shape)
            df = df.join(cc, how='left', on='SK_ID_CURR')
#            print(df.dtypes.value_counts())
            del cc
            gc.collect()

zeros = df.TARGET.value_counts(
    sort=True,
    ascending=False,
    dropna=True,
)[0]
ones = df.TARGET.value_counts(
    sort=True,
    ascending=False,
    dropna=True,
)[1]
print(f"{zeros:.2f}", f"{ones:.2f}")

Train samples: 1000, test samples: 1000
Application train test df shape: (2000, 235)
Bureau df shape: (205, 99)
Process bureau and bureau_balance - done in 0s
Previous applications df shape: (985, 221)
Process previous_applications - done in 0s
Pos-cash balance df shape: (989, 12)
Process POS-CASH balance - done in 0s
Installments payments df shape: (965, 26)
Process installments payments - done in 0s
Credit card balance df shape: (1000, 116)
Process credit card balance - done in 0s
Full model run - done in 1s
930.00 70.00


In [33]:
zo_sub = zeros/ones
oz_sub = ones/zeros
print(f"{zo_sub:.2f}", f"{oz_sub:.2f}")

13.29 0.08


In [34]:
if (zo/zo_sub >= 3/2 or zo/zo_sub <= 2/3):
    for _ in range(50):
        print("!")
    print(
        "L'imbalance des targets a été fortement modifiée par le",
        "subsampling. Elle est passée de",
        f"{zo:.2f}",
        "à",
        f"{zo_sub:.2f}",
        ".",
    )
else:
    print(
        "L'imbalance de",
        f"{zo:.2f}",
        "des targets du dataset est de",
        f"{zo_sub:.2f}",
        "après subsampling.",
    )

L'imbalance de 11.39 des targets du dataset est de 13.29 après subsampling.


In [35]:
1234567890123456789012345678901234567890123456789012345678901234567890123456789

1234567890123456789012345678901234567890123456789012345678901234567890123456789

# ee

In [36]:
#df.to_csv("df_agg.csv")

In [37]:
#df.to_csv("../input/df_agg.csv")

In [38]:
#df.to_csv("../df_agg.csv")

In [39]:
#df.to_csv("../working/df_agg.csv")

In [40]:
#pd.read_csv('../df_agg.csv')

In [41]:
%%script echo skipped
df = pd.read_csv("df_agg.csv")
print(df.shape)
df.columns

Couldn't find program: 'echo'


In [42]:
%%script echo skipped
for dirname, _, filenames in os.walk('..'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Couldn't find program: 'echo'


In [43]:
%%script echo skipped
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Couldn't find program: 'echo'


In [44]:
#!pwd

In [45]:
#!ls

In [46]:
#df.dtypes.value_counts()

In [47]:
#df.info()

In [48]:
#df.isna().mean()

# ee

In [49]:
cols_of_df = list(df.columns)

In [50]:
%%script echo skipped
for j in cols_of_df:
    df = df.rename(columns={j: re.sub(r"[ ]", r"_a_", j)})
    df = df.rename(columns={j: re.sub(r"[-]", r"_b_", j)})
    df = df.rename(columns={j: re.sub(r"[:]", r"_c_", j)})
    df = df.rename(columns={j: re.sub(r"[/]", r"_d_", j)})
    df = df.rename(columns={j: re.sub(r"[,]", r"_e_", j)})
    df = df.rename(columns={j: re.sub(r"[:]", r"_f_", j)})

Couldn't find program: 'echo'


In [51]:
df = df.rename(columns=lambda x: x.replace(" ", "_a_")\
                                  .replace("-", "_b_")\
                                  .replace(":", "_c_")\
                                  .replace("/", "_d_")\
                                  .replace(",", "_e_")\
                                  .replace(":", "_f_"))

# ee

In [52]:
%%script echo skipped
with timer("Run LightGBM with kfold"):
    feat_importance = kfold_lightgbm(df, num_folds=10, stratified=False, debug=False)

Couldn't find program: 'echo'


# ee

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
%%script echo mauvaise idée
#surtout pas de random split car TARGET a des NaN en plus des 0 ou 1
X = df.drop("TARGET", axis="columns")
y = df["TARGET"]
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.2, random_state=42)

Couldn't find program: 'echo'


In [55]:
1234567890123456789012345678901234567890123456789012345678901234567890123456789

1234567890123456789012345678901234567890123456789012345678901234567890123456789

In [56]:
train_df = df[df['TARGET'].notnull()]
X_train = train_df.drop("TARGET", axis="columns")
y_train = train_df["TARGET"]
test_df = df[df['TARGET'].isnull()]
X_test = test_df.drop("TARGET", axis="columns")
y_test = test_df["TARGET"]

In [57]:
y_train.value_counts()

0.0    930
1.0     70
Name: TARGET, dtype: int64

# ee

In [62]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
X_train_i = imputer.fit_transform(X_train)
X_test_i = imputer.fit_transform(X_test)

# ee

In [63]:
%%script echo skipped
from imblearn.under_sampling import RandomUnderSampler
undersampler = RandomUnderSampler(sampling_strategy='majority')
X_train_u, y_train_u = undersampler.fit_resample(X_train_i, y_train)

Couldn't find program: 'echo'


In [64]:
from imblearn.over_sampling import SMOTE
#oversampler = RandomOverSampler(sampling_strategy='minority')
oversampler = SMOTE()
X_train_o, y_train_o = oversampler.fit_resample(X_train_i, y_train)

# ee

In [58]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier

In [59]:
param_distributions = {
    'dummy': {'class_weight': [{0: 1, 1: 1}, {0: oz, 1: zo}]},
    'naive_bayes': {'class_weight': [{0: oz, 1: zo}]},
    'sgd': {'loss': ['hinge', 'log', 'squared_hinge', 'modified_huber'],
            'penalty': ['l2', 'l1', 'elasticnet'],
            'class_weight': [{0: oz, 1: zo}]},
    'knn': {'n_neighbors': range(1, 11),
            'weights': ['uniform', 'distance'],
            'class_weight': [{0: oz, 1: zo}]},
    'svc': {'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
            'C': [0.1, 1, 10, 100, 1000],
            'gamma': ['auto', 'scale'],
            'class_weight': [{0: oz, 1: zo}]},
    'random_forest': {'n_estimators': [10, 100, 1000],
                      'criterion': ['gini', 'entropy'],
                      'class_weight': [{0: oz, 1: zo}]},
    'xgb': {'booster': ['gbtree', 'gblinear', 'dart'],
            'learning_rate': [0.1, 0.01, 0.001],
            'max_depth': range(1, 11),
            'class_weight': [{0: oz, 1: zo}]},
    'catboost': {'iterations': [10, 100, 1000],
                 'learning_rate': [0.1, 0.01, 0.001],
                 'depth': range(1, 11),
                 'class_weight': [{0: oz, 1: zo}]},
    'adaboost': {'n_estimators': [10, 100, 1000],
                 'learning_rate': [0.1, 0.01, 0.001],
                 'class_weight': [{0: oz, 1: zo}]},
    'lightgbm': {'boosting_type': ['gbdt', 'dart', 'goss'],
                 'learning_rate': [0.1, 0.01, 0.001],
                 'num_leaves': range(10, 110, 10),
                 'class_weight': [{0: oz, 1: zo}]},
}

In [60]:
param_distributions = {
    'dummy': {},
    'naive_bayes': {},
    'sgd': {'loss': ['hinge', 'log', 'squared_hinge', 'modified_huber'],
             'penalty': ['l2', 'l1', 'elasticnet']},
    'knn': {'n_neighbors': range(1, 11),
            'weights': ['uniform', 'distance']},
    'svc': {'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
            'C': [0.1, 1, 10, 100, 1000],
            'gamma': ['auto', 'scale']},
    'random_forest': {'n_estimators': [10, 100, 1000],
                      'criterion': ['gini', 'entropy']},
    'xgb': {'booster': ['gbtree', 'gblinear', 'dart'],
            'learning_rate': [0.1, 0.01, 0.001],
            'max_depth': range(1, 11)},
    'catboost': {'iterations': [10, 100, 1000],
                 'learning_rate': [0.1, 0.01, 0.001],
                 'depth': range(1, 11)},
    'adaboost': {'n_estimators': [10, 100, 1000],
                 'learning_rate': [0.1, 0.01, 0.001]},
    'lightgbm': {'boosting_type': ['gbdt', 'dart', 'goss'],
                 'learning_rate': [0.1, 0.01, 0.001],
                 'num_leaves': range(10, 110, 10)},
}

In [61]:
param_distributions = {
    'dummy': {},
    'naive_bayes': {},
}

In [65]:
classifiers = {
    'dummy': DummyClassifier(),
    'naive_bayes': GaussianNB(),
    'sgd': SGDClassifier(),
    'knn': KNeighborsClassifier(),
    'svc': SVC(),
    'random_forest': RandomForestClassifier(),
    'xgb': XGBClassifier(),
    'catboost': CatBoostClassifier(),
    'adaboost': AdaBoostClassifier(),
    'lightgbm': LGBMClassifier(),
}

In [66]:
classifiers = {
    'dummy': DummyClassifier(),
    'naive_bayes': GaussianNB(),
}

In [67]:
from sklearn.pipeline import Pipeline

In [68]:
=r

SyntaxError: invalid syntax (870885351.py, line 1)

In [ ]:
pipe = Pipeline(steps=[(name, estimator) for name, estimator in classifiers.items()])

In [ ]:
rs_cv = RandomizedSearchCV(estimator=Pipeline(classifiers),
                           param_distributions=param_distributions,
                           n_iter=10,
                           cv=5,
                           scoring=['roc_auc', 'accuracy'],
                           refit='roc_auc',
                           random_state=42)

In [ ]:
#rs_cv.fit(X_train, y_train)
#rs_cv.fit(X_train_u, y_train_u)
rs_cv.fit(X_train_o, y_train_o)

for algorithm in classifiers.keys():
    print(f"Best parameters for {algorithm}: {rs_cv.best_params_[algorithm]}")
    print(f"Best AUC score for {algorithm}: {rs_cv.best_score_[algorithm]['roc_auc']:.3f}")
    print(f"Best accuracy score for {algorithm}: {rs_cv.best_score_[algorithm]['accuracy']:.3f}")

#best_algorithm = rs_cv.best_estimator_.keys()[0]
best_algorithm = rs_cv.best_estimator_.named_steps.keys()
print(f"Overall best algorithm: {best_algorithm}")
print(f"Best AUC score: {rs_cv.best_score_[best_algorithm]['roc_auc']:.3f}")
print(f"Best accuracy score: {rs_cv.best_score_[best_algorithm]['accuracy']:.3f}")

# ee

In [ ]:
clf1 = RandomForestClassifier(random_state=42)
clf2 = SVC(probability=True, random_state=42)
clf3 = LogisticRegression(random_state=42)
clf4 = DecisionTreeClassifier(random_state=42)
clf5 = KNeighborsClassifier()
clf6 = MultinomialNB()
clf7 = GradientBoostingClassifier(random_state=42)

In [ ]:
param1 = {}
param1['classifier__n_estimators'] = [10, 50, 100, 250]
param1['classifier__max_depth'] = [5, 10, 20]
param1['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param1['classifier'] = [clf1]

param2 = {}
param2['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param2['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param2['classifier'] = [clf2]

param3 = {}
param3['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param3['classifier__penalty'] = ['l1', 'l2']
param3['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param3['classifier'] = [clf3]

param4 = {}
param4['classifier__max_depth'] = [5,10,25,None]
param4['classifier__min_samples_split'] = [2,5,10]
param4['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param4['classifier'] = [clf4]

param5 = {}
param5['classifier__n_neighbors'] = [2,5,10,25,50]
param5['classifier'] = [clf5]

param6 = {}
param6['classifier__alpha'] = [10**0, 10**1, 10**2]
param6['classifier'] = [clf6]

param7 = {}
param7['classifier__n_estimators'] = [10, 50, 100, 250]
param7['classifier__max_depth'] = [5, 10, 20]
param7['classifier'] = [clf7]

In [ ]:
pipeline = Pipeline([('classifier', clf1)])
params = [param1, param2, param3, param4, param5, param6, param7]

In [ ]:
%%time
gs = GridSearchCV(pipeline, params, cv=3, n_jobs=-1, scoring='roc_auc').fit(X_train, y_train)
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
print("Test Precision:",precision_score(gs.predict(X_test), y_test))
print("Test Recall:",recall_score(gs.predict(X_test), y_test))
print("Test ROC AUC Score:",roc_auc_score(gs.predict(X_test), y_test))

In [ ]:
%%time
rs = RandomizedSearchCV(pipeline, params, cv=3, n_jobs=-1, scoring='roc_auc').fit(X_train, y_train)
rs.best_params_

In [ ]:
rs.best_score_

In [ ]:
print("Precision:",precision_score(rs.predict(X_test), y_test))
print("Recall:",recall_score(rs.predict(X_test), y_test))
print("ROC AUC Score:",roc_auc_score(rs.predict(X_test), y_test))

# ee

In [ ]:
=r on va faire un griudsearch fine tuné là

In [ ]:
y_pred = rs_cv.best_estimator_[best_algorithm].predict(X_test)